In [2]:
!pip -q install bitsandbytes streamlit pyngrok

In [1]:
!mkdir -p core utils ui

In [4]:
%%bash
touch core/__init__.py
touch utils/__init__.py
touch ui/__init__.py
#touch ui/tabs/__init__.py

  i n d o w s   S u b s y s t e m   f o r   L i n u x   h a s   n o   i n s t a l l e d   d i s t r i b u t i o n s . 
 Y o u   c a n   r e s o l v e   t h i s   b y   i n s t a l l i n g   a   d i s t r i b u t i o n   w i t h   t h e   i n s t r u c t i o n s   b e l o w : 
 
 U s e   ' w s l . e x e   - - l i s t   - - o n l i n e '   t o   l i s t   a v a i l a b l e   d i s t r i b u t i o n s 
 a n d   ' w s l . e x e   - - i n s t a l l   < D i s t r o > '   t o   i n s t a l l . 
 

CalledProcessError: Command 'b'touch core/__init__.py\ntouch utils/__init__.py\ntouch ui/__init__.py\n#touch ui/tabs/__init__.py\n'' returned non-zero exit status 1.

In [6]:
%%writefile core/__init__.py
# Core module initialization

Writing core/__init__.py


In [7]:
%%writefile ui/__init__.py
# UI module initialization

Writing ui/__init__.py


In [9]:
%%writefile utils/__init__.py
# Utils module initialization

Writing utils/__init__.py


In [10]:
%%writefile config.py
import os

MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.3"
PROJECT_OUTPUT_DIR = "generated_projects"
DEFAULT_PROJECT_STRUCTURE = {
    "folders": [
        "assets",
        "core",
        "utils",
        "ui"
    ],
    "files": {
        "main.py": "Placeholder for main application entry point.",
        "config.py": "Placeholder for project configuration.",
        "README.md": "Placeholder for project documentation.",
        "requirements.txt": "Placeholder for project dependencies.",
        "core/__init__.py": "# Makes 'core' a Python package",
        "core/logic.py": "Placeholder for core business logic.",
        "utils/__init__.py": "# Makes 'utils' a Python package",
        "utils/helpers.py": "Placeholder for utility functions.",
        "ui/__init__.py": "# Makes 'ui' a Python package",
        "ui/display.py": "Placeholder for UI or display logic."
    }
}

DEFAULT_LLM_PARAMS = {
    "temperature": 0.7,
    "max_new_tokens": 4096,
    "debug_mode": True
}

Writing config.py


In [11]:
%%writefile utils/session_state.py
import streamlit as st

def initialize_state():

    if 'project_plan' not in st.session_state:
        st.session_state.project_plan = None

    if 'generation_log' not in st.session_state:
        st.session_state.generation_log = []

    if 'project_zip_path' not in st.session_state:
        st.session_state.project_zip_path = None

    if 'generated_code' not in st.session_state:
        st.session_state.generated_code = {}

    if 'project_stats' not in st.session_state:
        st.session_state.project_stats = {
            "total_files": 0,
            "total_lines": 0,
            "time_taken": 0.0,
            "files_list": []
        }

Writing utils/session_state.py


In [12]:
%%writefile utils/file_manager.py
import os
import shutil
import zipfile
import streamlit as st
from config import PROJECT_OUTPUT_DIR

def create_project_structure(project_name, structure_plan):
 
    project_path = os.path.join(PROJECT_OUTPUT_DIR, project_name)

    if os.path.exists(project_path):
        shutil.rmtree(project_path)

    os.makedirs(project_path, exist_ok=True)

    log = []

    if "folders" in structure_plan:
        for folder in structure_plan["folders"]:
            folder_path = os.path.join(project_path, folder)
            os.makedirs(folder_path, exist_ok=True)
            log.append(f"Created folder: {folder_path}")

    if "files" in structure_plan:
        for file_path, description in structure_plan["files"].items():
            full_file_path = os.path.join(project_path, file_path)
            os.makedirs(os.path.dirname(full_file_path), exist_ok=True)
            with open(full_file_path, 'w') as f:
                if description.startswith("Placeholder"):
                    f.write(f"# {description}\n")
                else:
                     f.write("") 
            log.append(f"Created empty file: {full_file_path}")

    return project_path, log

def save_code_to_file(file_path, code):

    try:
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(code)
        return True, f"Successfully saved code to {file_path}"
    except Exception as e:
        return False, f"Error saving file {file_path}: {e}"

def zip_project_folder(project_path, project_name):
 
    zip_file_name = f"{project_name}.zip"
    zip_file_path = os.path.join(PROJECT_OUTPUT_DIR, zip_file_name)

    try:
        with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for root, dirs, files in os.walk(project_path):
                for file in files:
                    full_path = os.path.join(root, file)
                    relative_path = os.path.relpath(full_path, project_path)
                    zipf.write(full_path, relative_path)

        return zip_file_path, f"Project zipped successfully: {zip_file_path}"
    except Exception as e:
        return None, f"Error zipping project: {e}"

Writing utils/file_manager.py


In [13]:
%%writefile core/prompts.py
from config import DEFAULT_PROJECT_STRUCTURE

def get_planner_system_prompt():

    file_list = "\n".join([f"- {f}" for f in DEFAULT_PROJECT_STRUCTURE['files'].keys()])

    prompt = f"""
You are an expert software architect. Your job is to analyze a user's request and create a concise, one-sentence functional description for *each file* in a predefined project structure.

**Your Task:**
Based on the user's request, you must provide a new description for *every file* listed below.
Do not add, remove, or rename any files.
If a file is not needed for the user's request, you *must* describe it as 'Placeholder' or 'Basic boilerplate'.

**Project File Structure:**
{file_list}

**Response Format:**
You *must* respond *only* with a Python dictionary. The keys must be the filenames (e.g., 'main.py') and the values must be the new one-sentence descriptions.

**Example Response:**
{{
    "main.py": "Main entry point to run the application.",
    "config.py": "Stores configuration variables like API keys.",
    "README.md": "Basic boilerplate documentation.",
    "requirements.txt": "Placeholder. No external libraries needed.",
    "core/__init__.py": "# Makes 'core' a Python package",
    "core/logic.py": "Contains the core function to process the data.",
    "utils/__init__.py": "# Makes 'utils' a Python package",
    "utils/helpers.py": "Placeholder. No helper functions needed.",
    "ui/__init__.py": "# Makes 'ui' a Python package",
    "ui/display.py": "Contains the function to print the output."
}}
"""
    return prompt.strip()


def get_coder_system_prompt():

    prompt = """
You are an expert Python developer. Your *only* job is to write complete, correct, and production-ready code for a *single file*.

**Rules:**
1.  You *must* respond *only* with the raw code for the file requested.
2.  Do *not* write *any* explanations, apologies, or text before or after the code.
3.  Do *not* use Markdown (e.g., ```python ... ```).
4.  Ensure the code is complete and functional.
5.  If the task is 'Placeholder', just write a simple comment (e.g., '# This file is a placeholder.')
"""
    return prompt.strip()


def get_coder_user_prompt(project_plan, file_name):

    if file_name not in project_plan["files"]:
        return f"Error: File '{file_name}' not found in project plan."

    task_description = project_plan["files"][file_name]
    plan_context = "\n".join([f"- {f}: {desc}" for f, desc in project_plan["files"].items()])

    prompt = f"""
**Overall Project Plan:**
{plan_context}

---

**Your Specific Task:**
Write the *complete code* for the file: `{file_name}`
**File Description:** "{task_description}"
"""
    return prompt.strip()

Writing core/prompts.py


In [14]:
%%writefile core/model.py
import torch
import streamlit as st
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from config import MODEL_ID

@st.cache_resource
def load_model():

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    try:
        tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

        model = AutoModelForCausalLM.from_pretrained(
            MODEL_ID,
            quantization_config=bnb_config,
            torch_dtype=torch.float16,
            device_map="auto",
            trust_remote_code=True,
        )

        model.config.use_cache = False

        return model, tokenizer
    except Exception as e:
        st.error(f"Error loading model: {e}")
        return None, None

def generate_text(system_prompt, user_prompt, model, tokenizer, llm_params):

    if model is None or tokenizer is None:
        return "Error: Model not loaded."

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
    model_inputs = encodeds.to(model.device)
    input_token_length = model_inputs.shape[1]

    try:
        generated_ids = model.generate(
            model_inputs,
            max_new_tokens=llm_params.get("max_new_tokens", 4096),
            temperature=llm_params.get("temperature", 0.7),
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
        new_token_ids = generated_ids[0, input_token_length:]

        reply = tokenizer.decode(new_token_ids, skip_special_tokens=True)

        return reply.strip()

    except Exception as e:
        return f"Error during text generation: {e}"

Writing core/model.py


In [15]:
%%writefile core/agent.py
import os
import time
import ast
import re
import streamlit as st
from core.model import generate_text
from core.prompts import get_planner_system_prompt, get_coder_system_prompt, get_coder_user_prompt
from utils.file_manager import create_project_structure, save_code_to_file, zip_project_folder
from config import DEFAULT_PROJECT_STRUCTURE

def log_message(message, type="info", placeholder=None):

    st.session_state.generation_log.append(message)

    if placeholder:
        log_func = getattr(placeholder, type, placeholder.info)
        log_func(message)

def run_planner(user_request, model, tokenizer, llm_params, log_placeholder):

    log_message("Calling 'Planner' agent...", placeholder=log_placeholder)
    system_prompt = get_planner_system_prompt()

    response = generate_text(system_prompt, user_request, model, tokenizer, llm_params)

    if "Error" in response:
        log_message(response, "error", placeholder=log_placeholder)
        return None

    try:
        start_index = response.find('{')
        end_index = response.rfind('}')

        if start_index == -1 or end_index == -1:
            raise ValueError("Could not find a dictionary in the model's response.")

        plan_str = response[start_index : end_index + 1]
        plan_str = plan_str.replace("Placeholder", "'Placeholder'").replace("Basic boilerplate", "'Basic boilerplate'")
        project_plan = ast.literal_eval(plan_str)

        if not isinstance(project_plan, dict):
            raise ValueError("Planner did not return a valid dictionary structure.")

        full_plan = {
            "folders": DEFAULT_PROJECT_STRUCTURE["folders"],
            "files": project_plan
        }

        st.session_state.project_plan = full_plan
        log_message("✅ Project plan generated successfully.", "success", placeholder=log_placeholder)
        return full_plan

    except Exception as e:
        log_message(f"Error parsing planner response: {e}\nResponse was: {response}", "error", placeholder=log_placeholder)
        return None

def clean_code_output(raw_code):
    start_fence = raw_code.find("```")
    if start_fence == -1:
        return raw_code.strip()
    first_newline = raw_code.find('\n', start_fence)
    if first_newline == -1:
        code_start = start_fence + 3
    else:
        code_start = first_newline + 1
    last_fence = raw_code.rfind("```")
    if last_fence == -1 or last_fence <= code_start:
        return raw_code[code_start:].strip()
    return raw_code[code_start : last_fence].strip()


def run_coder_and_saver(project_path, project_plan, model, tokenizer, llm_params, log_placeholder):
 
    log_message("Calling 'Coder' agent for each file...", placeholder=log_placeholder)

    total_lines = 0
    generated_files_list = []
    files_to_generate = project_plan.get("files", {})
    if not files_to_generate:
        log_message("No files found in the project plan.", "warning", placeholder=log_placeholder)
        return

    st.session_state.generated_code = {}

    for file_name, task_description in files_to_generate.items():
        log_message(f"Generating code for: {file_name}...", placeholder=log_placeholder)

        system_prompt = get_coder_system_prompt()
        user_prompt = get_coder_user_prompt(project_plan, file_name)
        raw_output = generate_text(system_prompt, user_prompt, model, tokenizer, llm_params)
        generated_code = clean_code_output(raw_output)

        if "Error" in raw_output:
            log_message(f"Error generating code for {file_name}: {raw_output}", "error", placeholder=log_placeholder)
            continue

        if not generated_code and "Placeholder" not in task_description:
            log_message(f"Error generating code for {file_name}: Model returned empty string.", "error", placeholder=log_placeholder)
            continue

        full_file_path = os.path.join(project_path, file_name)
        success, message = save_code_to_file(full_file_path, generated_code)

        if success:
            lines = len(generated_code.split('\n'))
            total_lines += lines
            generated_files_list.append({"File": file_name, "Status": "✅ Generated", "Lines": lines})
            log_message(f"✅ Saved {file_name} ({lines} lines).", "success", placeholder=log_placeholder)
            st.session_state.generated_code[file_name] = generated_code
        else:
            log_message(message, "error", placeholder=log_placeholder)
            generated_files_list.append({"File": file_name, "Status": "❌ Error", "Lines": 0})

    return total_lines, generated_files_list


def run_generation_pipeline(user_request, project_name, model, tokenizer, llm_params, log_placeholder):

    start_time = time.time()
    st.session_state.generation_log = []
    st.session_state.project_plan = None
    st.session_state.project_zip_path = None
    st.session_state.generated_code = {}
    st.session_state.project_stats = {"total_files": 0, "total_lines": 0, "time_taken": 0.0, "files_list": []}
    project_plan = run_planner(user_request, model, tokenizer, llm_params, log_placeholder)
    if project_plan is None:
        return

    log_message("Creating project directory structure...", placeholder=log_placeholder)
    project_path, logs = create_project_structure(project_name, project_plan)
    for log in logs:
        st.session_state.generation_log.append(log) 
        log_message(log, "info", placeholder=log_placeholder) 
    log_message(f"✅ Project structure created at: {project_path}", "success", placeholder=log_placeholder)
    total_lines, files_list = run_coder_and_saver(project_path, project_plan, model, tokenizer, llm_params, log_placeholder)
    log_message("Zipping project folder for download...", placeholder=log_placeholder)
    zip_path, message = zip_project_folder(project_path, project_name)
    if zip_path:
        st.session_state.project_zip_path = zip_path
        log_message(f"✅ Project zipped successfully!", "success", placeholder=log_placeholder)
    else:
        log_message(message, "error", placeholder=log_placeholder)

    end_time = time.time()
    time_taken = end_time - start_time
    st.session_state.project_stats = {
        "total_files": len(files_list),
        "total_lines": total_lines,
        "time_taken": round(time_taken, 2),
        "files_list": files_list
    }
    log_message(f"--- Pipeline Finished in {time_taken:.2f} seconds ---", "success", placeholder=log_placeholder)
    log_placeholder.balloons()

Writing core/agent.py


In [16]:
%%writefile ui/sidebar.py
import streamlit as st
from config import DEFAULT_LLM_PARAMS, MODEL_ID

def draw_sidebar():

    st.sidebar.title("⚙️ Control Panel")
    st.sidebar.markdown("Configure the agent and LLM parameters.")
    st.sidebar.header("Project Settings")
    project_name = st.sidebar.text_input(
        "Project Name",
        value="my_awesome_project",
        help="The name of the folder for your generated project."
    )

    st.sidebar.header("LLM Parameters")
    st.sidebar.text_input("Model", value=MODEL_ID, disabled=True)

    temperature = st.sidebar.slider(
        "Temperature",
        min_value=0.0,
        max_value=1.0,
        value=DEFAULT_LLM_PARAMS["temperature"],
        step=0.05,
        help="Controls randomness. Lower is more deterministic, higher is more creative."
    )

    max_new_tokens = st.sidebar.slider(
        "Max New Tokens",
        min_value=512,
        max_value=8192,
        value=DEFAULT_LLM_PARAMS["max_new_tokens"],
        step=256,
        help="Maximum number of tokens the model can generate for each file."
    )

    st.sidebar.header("Agent Controls")
    debug_mode = st.sidebar.checkbox(
        "Debug Mode",
        value=DEFAULT_LLM_PARAMS["debug_mode"],
        help="Show extra debug information in the logs."
    )

    st.sidebar.header("Utility")
    if st.sidebar.button("Clear Session Cache"):
        st.session_state.clear()
        st.rerun()

    llm_params = {
        "temperature": temperature,
        "max_new_tokens": max_new_tokens,
        "debug_mode": debug_mode
    }

    return project_name, llm_params

Writing ui/sidebar.py


In [17]:
%%writefile ui/widgets.py
import streamlit as st
import pandas as pd
import os

def draw_status_monitor():

    st.subheader("Live Agent Monitor")
    log_container = st.container(height=300, border=True)

    if 'generation_log' not in st.session_state or not st.session_state.generation_log:
      
        log_container.info("Agent logs will appear here...")
      
        return

    for log_entry in reversed(st.session_state.generation_log):
        if log_entry.startswith("✅"):
            log_container.success(log_entry)
        elif log_entry.startswith("Error") or log_entry.startswith("❌"):
            log_container.error(log_entry)
        elif log_entry.startswith("---"):
            log_container.markdown(f"**{log_entry}**")
        else:
            log_container.info(log_entry)

def draw_code_viewer():

    st.subheader("Generated Code Viewer")

    generated_code = st.session_state.get('generated_code', {})

    if not generated_code:
        st.info("No code has been generated yet. Run the agent from the 'Run' tab.")
        return

    file_list = list(generated_code.keys())
    selected_file = st.selectbox("Select a file to view:", file_list)

    if selected_file:
        st.code(generated_code[selected_file], language="python")

def draw_project_stats_and_download():

    st.subheader("Project Stats")
    stats = st.session_state.get('project_stats', {})

    if not stats or stats["total_files"] == 0:
        st.info("No stats to display. Run the agent to generate a project.")
        return

    col1, col2, col3 = st.columns(3)
    col1.metric("Total Files", stats.get("total_files", 0))
    col2.metric("Total Lines of Code", stats.get("total_lines", 0))
    col3.metric("Time Taken (sec)", stats.get("time_taken", 0.0))

    st.subheader("Generated Files")
    files_df = pd.DataFrame(stats.get("files_list", []))
    st.dataframe(files_df, use_container_width=True)
    st.subheader("Download Project")
    zip_path = st.session_state.get('project_zip_path')

    if zip_path and os.path.exists(zip_path):
        with open(zip_path, "rb") as f:
            st.download_button(
                label="Download Project as .zip",
                data=f,
                file_name=os.path.basename(zip_path),
                mime="application/zip",
            )
    else:
        st.warning("No project ZIP file found. Please run the generation pipeline first.")

Writing ui/widgets.py


In [18]:
%%writefile ui/tabs_layout.py
import streamlit as st
from ui.widgets import draw_status_monitor, draw_code_viewer, draw_project_stats_and_download

def draw_tabs():

    tab1, tab2, tab3 = st.tabs([
        "📝 Plan & Monitor",
        "VIEW: Code & Files",
        "📈 Stats & Download"
    ])

    with tab1:
        st.header("Project Plan & Generation Status")

        col1, col2 = st.columns([1, 1])

        with col1:
            st.subheader("Project Plan")
            plan_container = st.container(height=300, border=True)
            plan = st.session_state.get('project_plan')

            if plan:
                plan_container.json(plan)
            else:
                plan_container.info("The project plan will appear here after generation.")

        with col2:
            draw_status_monitor()

    with tab2:
        st.header("Review Generated Code")
        draw_code_viewer()

    with tab3:
        st.header("Project Statistics & Download")
        draw_project_stats_and_download()

Writing ui/tabs_layout.py


In [19]:
%%writefile main.py
import streamlit as st
from utils.session_state import initialize_state
from ui.sidebar import draw_sidebar
from ui.tabs_layout import draw_tabs
from core.model import load_model
from core.agent import run_generation_pipeline

st.set_page_config(
    page_title="Synaptic Codex Forge",
    page_icon="🤖",
    layout="wide"
)
initialize_state()
project_name, llm_params = draw_sidebar()

st.title("🤖 Synaptic Codex Forge")
st.markdown("Powered by `mistralai/Mistral-7B-Instruct-v0.3`")

model_loaded = False
with st.spinner("Loading Mistral-7B model... This may take a moment."):
    model, tokenizer = load_model()

if model is None or tokenizer is None:
    st.error("Model failed to load. Please check your setup and network connection.")
else:
    st.success("Model loaded successfully!")
    model_loaded = True

st.header("What do you want to build?")
user_request = st.text_area(
    "Describe your project in detail:",
    height=150,
    placeholder="e.g., 'A Python app that uses Flask to serve a single 'Hello World' API endpoint...'"
)

if model_loaded:
    if st.button("🚀 Generate Project", type="primary", use_container_width=True):
        if not user_request:
            st.warning("Please describe your project first.")
        elif not project_name:
            st.warning("Please enter a project name in the sidebar.")
        else:
            st.subheader("Live Agent Monitor")
            log_placeholder = st.container(height=400, border=True)
            run_generation_pipeline(
                user_request,
                project_name,
                model,
                tokenizer,
                llm_params,
                log_placeholder  
            )
            st.rerun()
    st.divider()
draw_tabs()

Writing main.py


In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("34K1X9NzK5wXpXmklcPPrpoeqr3_65bWBoArzPwhAxVEhc96m")
!nohup streamlit run main.py --server.port 8501 &
public_url = ngrok.connect(8501)
public_url

nohup: appending output to 'nohup.out'


<NgrokTunnel: "https://unbridged-flakily-lavera.ngrok-free.dev" -> "http://localhost:8501">